In [1]:
import sys
import time
import pickle
import importlib
import numpy as np
import matplotlib.pyplot as plt
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [2]:
#setting the number of shots and Grover operators.
max_iter = 100
nbit = 10
# number_grover_list = [0, 1, 2, 4, 8, 16, 32] # , 64, 128, 256]  # list of number of Grover operators
number_grover_list = [5 * i for i in range(20)]
shots_list = [8192] * len(number_grover_list) # list of number of shots
if len(shots_list) != len(number_grover_list):
    raise Exception('The length of shots_list should be equal to the length of number_grover_list.')

In [3]:
with open("pkls/hists_shots8192.pkl", "rb") as f:
    hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [4]:
def make_hit_list(hist_list):
    hit_list = []
    strlen = len(list(hist_list[0].keys())[0])
    for k in range(len(hist_list)):
        hits = hist_list[k].get("0"*strlen, 0)
        hit_list.append(hits)
    return hit_list

In [5]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import QREM_Filter_1, QREM_Filter_2, QREM_Filter_3, QREM_Filter_4
import mthree
from qiskit.test.mock import FakeManhattan

In [6]:
ignis_mitigator_list = []
s_lm0_mitigator_list = []
lnp_mitigator_list = []
mooney_mitigator_list = []
nation_mitigator_list = []
for i in range(0, max_iter):
    ignis_mitigator_list.append(meas_fitters[i].filter)
    s_lm0_mitigator_list.append(QREM_Filter_1(nbit + 1, meas_fitters[i].cal_matrices))
    lnp_mitigator_list.append(QREM_Filter_4(nbit + 1, meas_fitters[i].cal_matrices))
    mooney_mitigator_list.append(QREM_Filter_3(nbit + 1, meas_fitters[i].cal_matrices))
    mit = mthree.M3Mitigation(FakeManhattan())
    mit.single_qubit_cals = meas_fitters[i].cal_matrices
    nation_mitigator_list.append(mit)
    if i % 10 == 0:
        print("size", i, "finished")
print("length of ignis_mitigator_list: ", len(ignis_mitigator_list))
print("length of s_lm0_mitigator_list: ", len(s_lm0_mitigator_list))
print("length of lnp_mitigator_list: ", len(lnp_mitigator_list))
print("length of mooney_mitigator_list: ", len(mooney_mitigator_list))
print("length of nation_mitigator_list: ", len(nation_mitigator_list))

size 0 finished
size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
size 70 finished
size 80 finished
size 90 finished
length of ignis_mitigator_list:  100
length of s_lm0_mitigator_list:  100
length of lnp_mitigator_list:  100
length of mooney_mitigator_list:  100
length of nation_mitigator_list:  100


In [7]:
raw_hit_list_list = []
for i in range(max_iter):
    raw_hit_list_list.append( make_hit_list(hist_list_list[i]) )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [8]:
with open("pkls/raw_hit_list_list.pkl", "wb") as f:
    pickle.dump(raw_hit_list_list, f)

In [ ]:
ignis_hit_list_list, ignis_time_list_list = [], []
for i, hist_list in enumerate(hist_list_list):
    ignis_hist_list, ignis_time_list = [], []
    for hist in hist_list:
        t1 = time.perf_counter()
        ignis_hist_list.append( ignis_mitigator_list[i].apply(hist) )
        t2 = time.perf_counter()
        ignis_time_list.append( t2 - t1 )
        print("hist", t2 - t1, "s")
    ignis_hit_list_list.append( make_hit_list(ignis_hist_list) )
    ignis_time_list_list.append( ignis_time_list )
    print("iter", i, "finished\n")

In [ ]:
with open("pkls/ignis_hit_list_list.pkl", "wb") as f:
    pickle.dump(ignis_hit_list_list, f)

In [9]:
lnp_hit_list_list, lnp_time_list_list = [], []
for i, hist_list in enumerate(hist_list_list):
    lnp_hist_list, lnp_time_list = [], []
    for hist in hist_list:
        lnp_hist_list.append( lnp_mitigator_list[i].apply(hist) )
        lnp_time_list.append( lnp_mitigator_list[i].times() )
    lnp_hit_list_list.append( make_hit_list(lnp_hist_list) )
    lnp_time_list_list.append( lnp_time_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [10]:
with open("pkls/lnp_hit_list_list.pkl", "wb") as f:
    pickle.dump(lnp_hit_list_list, f)
with open("pkls/lnp_time_list_list.pkl", "wb") as f:
    pickle.dump(lnp_time_list_list, f)

In [11]:
s_lm0_hit_list_list, s_lm0_time_list_list = [], []
for i, hist_list in enumerate(hist_list_list):
    s_lm0_hist_list, s_lm0_time_list = [], []
    for hist in hist_list:
        s_lm0_hist_list.append( s_lm0_mitigator_list[i].apply(hist) )
        s_lm0_time_list.append( s_lm0_mitigator_list[i].times() )
    s_lm0_hit_list_list.append( make_hit_list(s_lm0_hist_list) )
    s_lm0_time_list_list.append( s_lm0_time_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [12]:
with open("pkls/s_lm0_hit_list_list.pkl", "wb") as f:
    pickle.dump(s_lm0_hit_list_list, f)
with open("pkls/s_lm0_time_list_list.pkl", "wb") as f:
    pickle.dump(s_lm0_time_list_list, f)

In [13]:
mooney01_hit_list_list, mooney01_time_list_list = [], []
for i, hist_list in enumerate(hist_list_list):
    mooney01_hist_list, mooney01_time_list = [], []
    for hist in hist_list:
        mooney01_hist_list.append( mooney_mitigator_list[i].apply(hist, threshold = 0.1) )
        mooney01_time_list.append( mooney_mitigator_list[i].times() )
    mooney01_hit_list_list.append( make_hit_list(mooney01_hist_list) )
    mooney01_time_list_list.append( mooney01_time_list )
    print("iter", i, "finished")

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iteraccumulator is positive, we might even ignoring the necessal

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal

iter 22 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

accumulator is positive, we might even ignoring the necessal positeritive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 31 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

e necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might ev

...

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iter 61 finished
accumulator is positive, we might even ignoring

iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 70 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

 78accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the

itive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iter 87 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal pos

In [14]:
with open("pkls/mooney01_hit_list_list.pkl", "wb") as f:
    pickle.dump(mooney01_hit_list_list, f)
with open("pkls/mooney01_time_list_list.pkl", "wb") as f:
    pickle.dump(mooney01_time_list_list, f)

In [15]:
mooney001_hit_list_list, mooney001_time_list_list = [], []
for i, hist_list in enumerate(hist_list_list):
    mooney001_hist_list, mooney001_time_list = [], []
    for hist in hist_list:
        mooney001_hist_list.append( mooney_mitigator_list[i].apply(hist, threshold = 0.01) )
        mooney001_time_list.append( mooney_mitigator_list[i].times() )
    mooney001_hit_list_list.append( make_hit_list(mooney001_hist_list) )
    mooney001_time_list_list.append( mooney001_time_list )
    print("iter", i, "finished")

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal pos

iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 11 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

iter 27 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iter 28 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we mi

 36 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the

accumulator is positive, we might even ignoring the necessal positive values.
iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 45 finished
accumulator is positive, we might even ignoring

iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
 54 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iter 64 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal

iter 81 finished
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
iteraccumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even igno

accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal positive values.
accumulator is positive, we might even ignoring the necessal pos

In [16]:
with open("pkls/mooney001_hit_list_list.pkl", "wb") as f:
    pickle.dump(mooney001_hit_list_list, f)
with open("pkls/mooney001_time_list_list.pkl", "wb") as f:
    pickle.dump(mooney001_time_list_list, f)

In [17]:
nation_hit_list_list, nation_time_list_list = [], []
meas_layout = [j for j in range(nbit + 1)]
for i, hist_list in enumerate(hist_list_list):
    nation_hist_list, nation_time_list = [], []
    for hist in hist_list:
        mit_hist, info = nation_mitigator_list[i].apply_correction(hist,
                                                                   meas_layout, 
                                                                   distance=None,
                                                                   method='iterative',
                                                                   max_iter=25, 
                                                                   tol=1e-5,
                                                                   return_mitigation_overhead=True,
                                                                   details=True)
        nation_hist_list.append( mit_hist )
        nation_time_list.append( info )
    nation_hit_list_list.append( make_hit_list(nation_hist_list) )
    nation_time_list_list.append( nation_time_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [18]:
with open("pkls/nation_hit_list_list.pkl", "wb") as f:
    pickle.dump(nation_hit_list_list, f)
with open("pkls/nation_time_list_list.pkl", "wb") as f:
    pickle.dump(nation_time_list_list, f)